# Edward and Mack Final Assignment Machine Learning 445 

In [1]:
%pip install bayesian-optimization torch torchvision pandas seaborn matplotlib scikit-learn datasets

  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-19.0.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached xxhash-3.5.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached propcache-0.3.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (10 kB)
  Using ca

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from bayes_opt import BayesianOptimization as bo
import pandas as pd
import seaborn as sns
import time
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import psutil
import os


device = torch.device('mps' if torch.mps.is_available() else 'cpu')
print("Using device:", device)

#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


Using device: mps


## Optimizer

In [ ]:
pbounds = {
    "epochs": (3, 18),
    "num_inner_functions": (10, 35),
    "num_outer_functions": (10, 35),
    "num_control_points": (2, 9),
    "range_min": (-5.0, -0.1),
    "range_max": (0.1, 5),
    "degree": (2,10),
    "lr": (1e-4, 1e-3)
}

